## Imports & Setup

In [ ]:
import sys
import yaml
import shutil
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from datetime import datetime

# Add the scripts directory to system path so we can import our modules
# Assumes notebook is in /notebooks and scripts are in /scripts
current_dir = Path.cwd()
sys.path.append(str(current_dir.parent / "scripts"))

# Import our custom robust modules
from project_paths import ROOT_DIR, CONFIG_DIR, DATA_DIR, resolve_path
import model_builder

print(f"✅ Project Root: {ROOT_DIR}")
print(f"✅ TensorFlow Version: {tf.__version__}")
print(f"✅ Devices: {tf.config.list_physical_devices()}")

## Experiment Manager

In [ ]:
def setup_experiment(config, config_source_path):
    """
    Creates a unique run directory and backs up the config.
    Returns: run_dir (Path), log_dir (Path), checkpoint_path (Path)
    """
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    run_name = config['project']['run_name']
    
    # Create runs/YYYYMMDD_HHMMSS_run_name
    run_dir = ROOT_DIR / "runs" / f"{run_name}_{timestamp}"
    run_dir.mkdir(parents=True, exist_ok=True)
    
    # Create subdirectories
    log_dir = run_dir / "logs"
    log_dir.mkdir(exist_ok=True)
    
    checkpoint_dir = run_dir / "checkpoints"
    checkpoint_dir.mkdir(exist_ok=True)
    
    # Backup the config file
    shutil.copy(config_source_path, run_dir / "config.yaml")
        
    print(f"🚀 Experiment initialized at: {run_dir}")
    return run_dir, log_dir, checkpoint_dir

# Load the config
config_path = CONFIG_DIR / "model_config.yaml"
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

# Setup the run
run_dir, log_dir, checkpoint_dir = setup_experiment(config, config_path)

## Data Loading Pipeline

In [ ]:
# Resolve the dataset path using our helper (handles absolute/relative)
dataset_path = resolve_path(config['data']['dataset_path'])

img_size = (config['data']['img_height'], config['data']['img_width'])
batch_size = config['data']['batch_size']
color_mode = 'grayscale' if config['data']['channels'] == 1 else 'rgb'

print(f"📂 Loading data from: {dataset_path}")

# 1. Load Training Data
train_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    validation_split=config['data']['validation_split'],
    subset="training",
    seed=config['project']['seed'],
    image_size=img_size,
    batch_size=batch_size,
    color_mode=color_mode
)

# 2. Load Validation Data
val_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    validation_split=config['data']['validation_split'],
    subset="validation",
    seed=config['project']['seed'],
    image_size=img_size,
    batch_size=batch_size,
    color_mode=color_mode
)

# 3. Performance Optimization (The "Secret Sauce" for speed)
# cache() keeps images in memory after the first epoch
# prefetch() prepares the next batch while the GPU/CPU is working
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

print("✅ Data Pipeline Ready")

## Build & Compile the Model

In [ ]:
# Build the model dynamically from the config
model = model_builder.build_model_from_config(config)

model.summary()

## Callbacks and Training

In [ ]:
# Define Callbacks
callbacks = [
    # Stop if validation loss doesn't improve for 'patience' epochs
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=config['train']['callbacks']['early_stopping_patience'],
        restore_best_weights=True,
        verbose=1
    ),
    # Save the best model to the run folder
    tf.keras.callbacks.ModelCheckpoint(
        filepath=str(checkpoint_dir / "best_model.keras"),
        monitor='val_loss',
        save_best_only=True,
        verbose=1
    ),
    # Log metrics for visualization
    tf.keras.callbacks.TensorBoard(log_dir=log_dir),
    
    # Save CSV logs for easy plotting later
    tf.keras.callbacks.CSVLogger(str(run_dir / "training_log.csv"))
]

# --- NEW: Add Scheduler if enabled in config ---
if config['train']['callbacks'].get('use_lr_scheduler', False):
    scheduler = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=config['train']['callbacks']['reduce_lr_factor'],
        patience=config['train']['callbacks']['reduce_lr_patience'],
        min_lr=config['train']['callbacks']['reduce_lr_min'],
        verbose=1  # It will print "Reducing learning rate to..."
    )
    callbacks.append(scheduler)
    print("✅ Learning Rate Scheduler: ENABLED")

# START TRAINING
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=config['train']['epochs'],
    callbacks=callbacks
)

# Save the final model just in case
model.save(run_dir / "final_model.keras")
print("🏁 Training Complete!")

## Visualization

In [ ]:
import math

def plot_history(history):
    # 1. Identify unique base metrics (ignore 'val_' prefix)
    # We filter keys to find the "training" versions (e.g., 'loss', 'accuracy', 'precision')
    all_keys = history.history.keys()
    base_metrics = [k for k in all_keys if not k.startswith('val_')]
    
    # 2. Determine Grid Size
    n_metrics = len(base_metrics)
    n_cols = 2  # Two plots per row looks best
    n_rows = math.ceil(n_metrics / n_cols)
    
    # 3. Create Figure
    # Scale height based on number of rows
    plt.figure(figsize=(12, 4 * n_rows))
    
    epochs_range = range(len(history.history[base_metrics[0]]))

    for i, metric in enumerate(base_metrics):
        plt.subplot(n_rows, n_cols, i + 1)
        
        # Plot Training Metric
        plt.plot(epochs_range, history.history[metric], label=f'Training {metric.capitalize()}')
        
        # Plot Validation Metric (if it exists)
        val_key = f"val_{metric}"
        if val_key in all_keys:
            plt.plot(epochs_range, history.history[val_key], label=f'Validation {metric.capitalize()}')
            
        plt.legend(loc='best')
        plt.title(f'Training and Validation {metric.capitalize()}')
        plt.xlabel('Epochs')
        
        # Only show grid and ticks for cleaner look
        plt.grid(True, alpha=0.3)
        # Reduce x-ticks density if many epochs
        if len(epochs_range) > 20:
             plt.xticks(epochs_range[::5]) # Show every 5th epoch
        else:
             plt.xticks(epochs_range[::2])

    plt.tight_layout()
    plt.show()

# Usage
plot_history(history)